Environment setup.

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
colab_data_path = "/content/drive/MyDrive/Seminar2/data/ribe_512x768/"
colab_dir = "/content/drive/MyDrive/Seminar2/model/"
model_name_or_path = "google/vit-base-patch16-224-in21k"

Imports.

In [2]:
import numpy as np
import torch
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets
from transformers import (
    ViTForImageClassification,
    ViTFeatureExtractor,
    Trainer,
    TrainingArguments,
)
from transformers.trainer_utils import EvalPrediction
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

Prepare dataset splits.

In [ ]:
dataset = load_dataset("imagefolder", data_dir=colab_data_path)
splits = dataset["train"].train_test_split(test_size=0.33)
dataset["train"] = splits["train"]
dataset["val"] = splits["test"]

Define image augmentations.

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
])

Apply the transforms to the train set. Concatenate the transformed and original train set.

In [ ]:
transformed_train_dataset = dataset["train"].map(
    lambda example: {"image": transform(example["image"]), "label": example["label"]}
)

dataset["train"] = concatenate_datasets([transformed_train_dataset, dataset["train"]])

Configure the Feature Extractor and apply it to the dataset.

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_name_or_path,
    do_resize=False,
    do_rescale=False,
    patch_size=64,
)

def transform(example_batch):
    inputs = feature_extractor(
        [x.convert("RGB") for x in example_batch["image"]], return_tensors="pt"
      )
    inputs["labels"] = example_batch["label"]
    return inputs

dataset = dataset.with_transform(transform)
labels = dataset['train'].features['label'].names

Configure Vision Transformer model.

In [ ]:
# Set interpolate_pos_encoding=True in the source code.
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,
)

model.config.hidden_dropout_prob = 0.5

Define auxiliary functions for the training procedure and configure training parameters.

In [ ]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

def compute_metrics(eval_pred: EvalPrediction):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return {
    "acc": accuracy_score(eval_pred.label_ids, preds),
    "f1": f1_score(eval_pred.label_ids, preds, average="weighted"),
    "precision": precision_score(eval_pred.label_ids, preds, average="weighted"),
    "recall": recall_score(eval_pred.label_ids, preds, average="weighted")
    }

In [ ]:
training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/Seminar2/model",
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=6,
  fp16=True,
  save_steps=60,
  eval_steps=60,
  warmup_steps=500,
  logging_steps=60,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  weight_decay=0.1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=feature_extractor,
)

Train the model.

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Evaluate the model.

In [ ]:
metrics = trainer.evaluate(dataset["test"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)